# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,1.33,41,40,7.00,RU,2024-11-11 22:14:48
1,1,horta,38.5333,-28.6333,20.57,95,100,7.60,PT,2024-11-11 22:19:38
2,2,port-aux-francais,-49.3500,70.2167,5.18,93,100,9.63,TF,2024-11-11 22:19:40
3,3,camana,-16.6228,-72.7111,18.12,87,77,2.08,PE,2024-11-11 22:19:41
4,4,serdar,38.9764,56.2757,10.29,83,100,0.99,TM,2024-11-11 22:19:42


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

import pandas as pd
import hvplot.pandas

# Assuming city_data_df is already defined and contains the necessary data
# Ensure that the DataFrame has columns for 'Lat', 'Lng', and 'Humidity'

# Create the map with color coding based on city names
city_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity',  # Size of the points based on humidity
    color='City',  # Color coding based on city names
    frame_width=800, 
    frame_height=600,
    title='City Humidity Map',
    cmap='Category10'  # You can choose a categorical color map
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Define your ideal weather conditions
ideal_temperature_min = 20  # Minimum ideal temperature in Fahrenheit
ideal_temperature_max = 100  # Maximum ideal temperature in Fahrenheit
max_humidity = 80            # Maximum ideal humidity percentage
max_cloudiness = 50          # Maximum ideal cloudiness percentage
max_wind_speed = 15          # Maximum ideal wind speed in mph

# Narrow down cities that fit criteria
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temperature_min) &
    (city_data_df['Max Temp'] <= ideal_temperature_max) &
    (city_data_df['Humidity'] <= max_humidity) &
    (city_data_df['Cloudiness'] <= max_cloudiness) &
    (city_data_df['Wind Speed'] <= max_wind_speed)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Select the relevant columns to match the intended output format
ideal_weather_df = ideal_weather_df[['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]

# Limit to 20 rows
ideal_weather_df = ideal_weather_df.head(20)

# Display sample data
if not ideal_weather_df.empty:
    print(ideal_weather_df)  # Display the filtered DataFrame
else:
    print("No cities meet the ideal weather conditions.")

     City_ID           City      Lat       Lng  Max Temp  Humidity  \
12        12        diapaga  12.0708    1.7889     20.54        23   
21        21          dubbo -32.2500  148.6167     29.07        32   
35        35     georgetown   5.4112  100.3354     30.53        72   
37        37  blackmans bay -43.0167  147.3167     21.07        56   
44        44          numan   9.4667   12.0333     22.29        38   
48        48         gebeit  21.0667   36.3167     20.94        73   
53        53       freetown   8.4840  -13.2299     26.36        77   
60        60     monkey bay -14.0728   34.9201     27.70        60   
61        61      waraseoni  21.7500   80.0333     26.56        47   
72        72      vrindavan  27.5833   77.7000     29.20        35   
73        73         avarua -21.2078 -159.7750     23.03        64   
83        83  atafu village  -8.5421 -172.5159     28.74        72   
89        89          saijo  33.9167  133.1833     20.76        71   
92        92        

### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Assuming ideal_weather_df is already defined and contains the relevant columns
# Create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Optionally, if you want to keep the same number of rows as in ideal_weather_df (which should be 20)
# Ensure that hotel_df contains the same 20 cities
hotel_df = hotel_df.head(20)  # This is optional if ideal_weather_df already has 20 rows

# Display the entire hotel_df to verify the output
print(hotel_df)  # Display the DataFrame

              City Country      Lat       Lng  Humidity Hotel Name
12         diapaga      BF  12.0708    1.7889        23           
21           dubbo      AU -32.2500  148.6167        32           
35      georgetown      MY   5.4112  100.3354        72           
37   blackmans bay      AU -43.0167  147.3167        56           
44           numan      NG   9.4667   12.0333        38           
48          gebeit      SD  21.0667   36.3167        73           
53        freetown      SL   8.4840  -13.2299        77           
60      monkey bay      MW -14.0728   34.9201        60           
61       waraseoni      IN  21.7500   80.0333        47           
72       vrindavan      IN  27.5833   77.7000        35           
73          avarua      CK -21.2078 -159.7750        64           
83   atafu village      TK  -8.5421 -172.5159        72           
89           saijo      JP  33.9167  133.1833        71           
92           wajir      KE   1.7471   40.0573        68       

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
import requests

# Set parameters for the hotel search
radius = 10000  # Radius in meters
base_url = "https://api.geoapify.com/v2/places"  # Geoapify API base URL

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Define the parameters for the API request
    params = {
        "categories": "accommodation.hotel",  # Updated category to explicitly specify hotels
        "apiKey": geoapify_key,              # Your Geoapify API key
        "limit": 1,                          # Limit to 1 result
        "filter": f"circle:{longitude},{latitude},{radius}",  # Circle filter for proximity
        "bias": f"proximity:{longitude},{latitude}"           # Bias results by proximity
    }

    try:
        # Make an API request using the params dictionary
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Convert the API response to JSON format
        response_json = response.json()

        # Grab the first hotel from the results and store the name in the DataFrame
        if response_json.get("features"):  # Check if features exist in the response
            hotel_name = response_json["features"][0]["properties"].get("name", "No name available")
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    except requests.exceptions.RequestException as e:
        # Handle any request-related errors
        print(f"Error fetching data for {row['City']} ({latitude}, {longitude}): {e}")
        hotel_df.loc[index, "Hotel Name"] = "API error"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated DataFrame with hotel information
print("Hotel search completed.")
hotel_df.head()  # Display sample data


Starting hotel search
diapaga - nearest hotel: Le Gacilien
dubbo - nearest hotel: Blue Gum Motel
georgetown - nearest hotel: Page 63 hostel
blackmans bay - nearest hotel: Villa Howden
numan - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
freetown - nearest hotel: Formal Leona hotel
monkey bay - nearest hotel: Chimanga Resthouse
waraseoni - nearest hotel: No hotel found
vrindavan - nearest hotel: Dwarka Kunj
avarua - nearest hotel: Paradise Inn
atafu village - nearest hotel: No hotel found
saijo - nearest hotel: 玉の家西条
wajir - nearest hotel: Kilimanjaro
jinghong - nearest hotel: No name available
new norfolk - nearest hotel: Woodbridge on the Derwent
tanjung sepat - nearest hotel: Hocks house
launceston - nearest hotel: Hotel Charles
tezu - nearest hotel: No hotel found
beihai - nearest hotel: Pearl Bay Seaview Hotel
Hotel search completed.


,City,Country,Lat,Lng,Humidity,Hotel Name
12,diapaga,BF,12.0708,1.7889,23,Le Gacilien
21,dubbo,AU,-32.2500,148.6167,32,Blue Gum Motel
35,georgetown,MY,5.4112,100.3354,72,Page 63 hostel
37,blackmans bay,AU,-43.0167,147.3167,56,Villa Howden
44,numan,NG,9.4667,12.0333,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
import pandas as pd
import hvplot.pandas

# Check if the required columns exist in the DataFrame
required_columns = {'Lng', 'Lat', 'Humidity', 'City', 'Country', 'Hotel Name'}
if not required_columns.issubset(hotel_df.columns):
    raise ValueError(f"Missing required columns in hotel_df. Expected columns: {required_columns}")

# Configure the map plot
city_map = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity',  # Size of the points based on humidity
    color='City',  # Color coding based on city names
    frame_width=800, 
    frame_height=600,  # Ensure all parameters are properly closed
    title='City Humidity Map',
    cmap='Category10',  # Use a categorical color map
    hover_cols=['City', 'Country', 'Hotel Name']  # Add hover information
)

# Display the map
from IPython.display import display
display(city_map)



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)